In [435]:
%reload_ext autoreload
%autoreload 2
import os, sys
import logging
import numpy as np, pandas as pd
import config, rules
from utilities import *
from _rulesbuilding import *

In [436]:
rules = pd.read_csv(config.files['service_rules'])
rules = rules[(rules['text_match'].notna()) & (rules['text_exclude'].notna() == False)]

regexer = GroupedDataRegex('|')

matchers = []
rdecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'M', '(', ')'))
sdecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'S', '^(', ').*$'))
adecorator = matchers.append(GroupedDataRegexDecorator(regexer, 'A', '^.*(', ').*$'))

regexes = {}
for matcher in matchers:
    type = rules['matching']
    regex = rules[rules['matching'] == matcher.getType()][['service_id','text_match']]
    regex = matcher.make(regex.groupby('service_id')['text_match']).to_dict()
    inv_map = {v: k for k, v in regex.items()}
    regexes = {**regexes, **inv_map}

In [437]:
signals = {'^.*(' + ('|'.join(v)) + ').*$' : k for k, v in config.signals.items()}

In [438]:
data = pd.read_csv(config.files['raw'])

In [439]:
data = data[data['status'] != 'D']

In [440]:
data['service_id'] = data[['description']]
data['service_id'] = data[['service_id']].replace({'service_id':regexes}, regex=True)

In [441]:
data['signal_type'] = data[['description']]
data['signal_type'] = data[['signal_type']].replace({'signal_type': signals}, regex = True)

In [442]:
data = data.sort_values('last_updated').drop_duplicates('item_id',keep='last')

In [443]:
data = data.sort_values('last_updated').drop_duplicates('item_id',keep='last')

In [444]:
save_file(config.files['processed'], data.to_csv())

In [445]:
# print(data.to_string())